<a href="https://colab.research.google.com/github/UlaG0706/cwiczenie-wspolne-repo/blob/main/Time_to_Practice__EDA_0_with_partial_solutions_moj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img height="180px" src="https://drive.google.com/uc?export=view&id=141XOz6N4nk8Ru1sAl7vOsAToCLrSFCAX" alt="SDA logo" align="left" hspace="30px" vspace="50px"/>

# Welcome to your next notebook with SDA!

During the classes we will mostly use [Google Colaboratory](https://colab.research.google.com/?hl=en) which is a free Jupyter notebook environment that requires no setup and runs entirely in the cloud.

However, for bigger projects, especially involving Deep Learning and/or big data reading, it might be a better choice to setup Jupyter Notebook or Jupyter Lab on your computer. Also, it is worth noticing that there is a great number of useful extensions (see [nbextensions](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/index.html) and [jupyter-labextension](https://jupyterlab.readthedocs.io/en/stable/user/extensions.html)) not available for Colab users.

<img src="https://drive.google.com/uc?export=view&id=1UO2urRciECzoKE_vHy4RMGfFbkOWOGlW" alt="SDA logo" align="left" width="100px" hspace="10px" vspace="10px"/>

# Exploratory Data Analysis
## Data Cleaning and Preparation

Data understanding - collecting specific data needed to perform the following tasks:
* solve the problem
* provide data description (sometimes also labeling),
* exploration and quality verification.

Data types - `.csv` sheets (these are the ones we will be working on today), text, image, sound, video, weather data, stock data, public archives, etc.

The most important task - FEATURE EXTRACTION (feature engineering). A lot depends on the size of the set and the number of features - depending on these parameters, we will take different steps in the data analysis process.

#### **<font color='#306998'>Download </font><font color='#ffd33b'>chipotle.csv</font>**

Download `chipotle.tsv` file from the following kaggle site:
https://www.kaggle.com/datasets/navneethc/chipotle

Upload it to your Google Drive, mount it and load the data.

*Hint: you can copy the path to the file or change the working directory and then use just the file name with .tsv extenstion.*

In [6]:
import pandas as pd

In [20]:
url = 'https://raw.githubusercontent.com/matzim95/ML-datasets/master/chipotle.tsv'

In [8]:
#from google.colab import drive
#drive.mount('/content/drive')

In [9]:
import pandas as pd

In [22]:
# nie chce sie laczyc z dyskiem dlatego nie wywoluje tego
chipotle = pd.read_csv("/content/drive/MyDrive/SDA Data Science/files/chipotle.tsv", sep="\t")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/SDA Data Science/files/chipotle.tsv'

In [19]:
chipotle['item_price [$]'] = chipotle["item_price"].str.replace("$", "", regex=False).astype(float)
chipotle = chipotle.drop(columns=["item_price"])
chipotle.head()

NameError: name 'chipotle' is not defined

In [18]:
chipotle.dtypes

NameError: name 'chipotle' is not defined

### Dealing with missing data

Missing data is a fairly common problem faced by Data Analysts and Data Scientist. There are two main approaches:
* `df.dropna()` method - removes all rows with empty data from the table
* `df.fillna(value)` method- fills (or imputes) the data with the specified value

Later on you will also know some more advanced methods to impute missing data. What are your thoughts? What number should be given to the `<value>` to fill in missing observations?

In [ ]:
len(chipotle) #sprawdzam ile wgl mam obserwacji

In [ ]:
chipotle.isna().sum() # tu sprawdzamy ile braków jest

In [ ]:
chipotle['choice_description'].fillna("No description")

Incomplete values - how to deal with them:
* ignore them #tego sie raczej nie uzywa
* insert the value "unknown" #oznaczenie porpostu ze to brak danych -> np mamy mieszkania i pietra i mamy mieszkania gdzie nie mamy przypisanego pietra. za pomoca jakiej wartosci oznaczymy bez zmieniania typu danych ze pietro nie ostnieje. mozna wziac np liczbe bardzo wysoka 99, (o ile nie sa to mieszkania w dubaju :P , albo -1 np, 0 nie bo sa mieszkania na parterze. Chodzi o to zeby dac taka wartosc ktora nie wystepuje w danych)
* manually fill in on the basis of accumulated knowledge
* delete records with incomplete data
* complete algorithmically
    * look for the nearest neighbor
    * take the average
    * insert random values
    * create ML algorithm

In [ ]:
threshold = 0.2 #to jest procent danych ktorych brakuje

chipotle.columns[chipotle.isnull().mean() > threshold]

In [ ]:
# drop columns with more than 20% of missing data pozbywamy sie wgl kolumny gdzie brak danych jest wiekszy niz 20%
chipotle[chipotle.columns[chipotle.isnull().mean() <= threshold]]

In [ ]:
# drop rows with at least 20% of missing data (here one value) mniej hardcorowe podejscie - pozbywamy sie tylko wierszy gdzie w kolumnie sa braki danych, ale pozbywamy sie tez wartosci w pozostalych kolumnach gdzie choice description ma braki. wiec trezba zdecydowac czy faktycznie az tak nic nam nie mowia
chipotle.loc[chipotle.isnull().mean(axis=1) < threshold] # wyrzucam wiersze wszystkie gdzie sa w ktorejkolwiek kolumnie braki wieksze niz 20%

Artificially, generate more missing values:

In [ ]:
import numpy as np

chipotle['item_price [$]'] = chipotle['item_price [$]'].replace(8.75, np.NaN) # tutaj dla danej item price zastepujemy wartosc brakiem danej NaN - ale to tylko dla cwiczenia bo nielogiczne generalnie zeby dane wywalac

In [ ]:
chipotle.isna().sum() #no i tu widzimy ze jest az 730 item price gdzie brak daych

In [ ]:
chipotle.fillna(0) #wszystkie braki danych zastepujemy 0 takze ponizej opiuje i zastepuje tylko itemprice

In [ ]:
chipotle["item_price [$]"].fillna(0)

In [ ]:
chipotle.fillna(chipotle['item_price [$]'].median()) #tu wszystkie braki danych zastepujemy za pomoca mediany zmiennej item price

In [ ]:
chipotle['choice_description'].fillna(chipotle['item_price [$]'].median())

In [ ]:
# missing categorical features replaced the most frequent category
chipotle.fillna(chipotle['choice_description'].value_counts().idxmax())

In [ ]:
chipotle['choice_description'].fillna(chipotle['choice_description'].value_counts().idxmax())

## Data preprocessing / cleaning (noise removal) odstające danych. widac ze najczesciej przyjmuje wartosci 8-12 ale nagle jest jakas wartosc bardzo odchylona co moze bardzo zaburzac analize, korelacje moze zmienic i zaburza caly porces. JEsli tych obserwacji jest duzo to nie mozna stwierdzic ze zakloca trend ale jest jest jak na histogramie to najlepiej uptliera sie pozbyc, po sprawdzeniu czy jest napewno outlierem

<img src="https://drive.google.com/uc?export=view&id=19yZ4fZuosFbjGLp_yCRS0mSKBSFv0v0Z" alt="Data processing: outliers" title="Data processing: outliers" align="left" width="600px" hspace="20px" vspace="20px"/>

Remove outliers with standard deviation.

In [ ]:
#reguła 3 sigm(3 odchyleń standardowych) do sprawdzenia czy napewno jest outlierem -> zakladajac ze nasza zmienna ma rozklad normalny lub do niego zblizony to jesli przesuway sie od sredniej na prawo i lewo to 66% obserwacij powinno spelniac to odchylenie, o 2 sigmy 30pare, a 3 sigmy w prawo i lewo od sredniej to 99% wartosci. jesli cos jest poza tym przedzialem to na 99% jest to wartsc odstająca.
# dropping outlier observations (ie. rows) based on standard deviation
#3 odchylenia
factor = 3

item_price_mean = chipotle['item_price [$]'].mean()
item_price_std = chipotle['item_price [$]'].std()

upper_lim = item_price_mean + item_price_std * factor
lower_lim = item_price_mean - item_price_std * factor

#mozemy dodac tez warunek ze brak danych zostaje
mask = (chipotle['item_price [$]'] < upper_lim) & (chipotle['item_price [$]'] > lower_lim) | (chipotle['item_price [$]'] > lower_lim)


chipotle[mask]

In [ ]:
item_price_std

In [ ]:
# pay attention to missing data!
chipotle[~mask]

Removing outliers with percentiles:

In [ ]:
upper_lim = chipotle['item_price [$]'].quantile(.95)
lower_lim = chipotle['item_price [$]'].quantile(.05)

chipotle[(chipotle['item_price [$]'] < upper_lim) & (chipotle['item_price [$]'] > lower_lim)]

Limiting outliers instead of deleting - esp. important for small datasets (for large datasets, removing redundant information is not a problem).

In [ ]:
# w tym przypadku zawsze bedziemy mieli outliery. z tym rozwiazaniem ostrozeni, bo przewazeni usuwamy duzo wiecej obserwacji
upper_lim = chipotle['item_price [$]'].quantile(.95)
lower_lim = chipotle['item_price [$]'].quantile(.05)

chipotle.loc[(chipotle['item_price [$]'] > upper_lim), 'item_price [$]'] = upper_lim #zamiastusuwania outlierow to mowimy ze to co nim jest to ustawiam jego wartosc jako wartość maxymalna czyli upperlim (gdy przekracza uppr_im czyli granice, lub wartos minimalna gdy jest ponizej lower_lim)
chipotle.loc[(chipotle['item_price [$]'] < lower_lim), 'item_price [$]'] = lower_lim

In [ ]:
upper_lim #podejscie bardziej konserwatywne

In [ ]:
lower_lim

In [ ]:
chipotle['item_price [$]'].value_counts()

### Standardization

Standardization consists in standardizing the various data formats in a set:
* language coding style: UTF-8, ANSI etc.,
* Dates: Sep 12, 2019, Sep 12, 2019, 120919 etc.,
* New York City, NY, Z. New York,
* MIT, Massachusetts Institute of Technology
* Unrecognizable signs.


### Discretization
np mamy analize krajow i chcemy to rozbic tylko na kontynenty wiec kategoryzujemy, hiszpanie i wlochy wrzucamy do Europy a chile i brazylie do ameryki pld - uogolnianiae, zmniejszamy zmiennosc zmiennej w przypadku wartosci

Continuous data distribution is bad for most machine learning algorithms.

<img src="https://drive.google.com/uc?export=view&id=1O8fuKpYz3hR2Ab6d7SOUZLrvQWqDzM55" alt="Data processing: discretization" title="Data processing: discretization" align="right" width="300px" hspace="20px" vspace="20px"/>

> numerical binning example

| Value  | Bin  |
| ------ | ---- |
| 0-30   | Low  |
| 31-70  | Mid  |
| 71-100 | High |

> categorical binning example

| Value  | Bin  |
| ------ | ---- |
| Spain  | Europe |
| Italy  | Europe |
| Chile  | South America |
| Brazil | South America |



### Logarithmic transformation

* helps to deal with skewed data
* organizes the size of the data
* reduces the impact of outliers
* increases the reliability of the model
* negative values need to be dealt with



In [ ]:
(chipotle['item_price [$]'] + 1).transform(np.log)

In [ ]:
value =-5
value-value+1

In [ ]:
chipotle['item_price [$]'].min()

In [ ]:
# if there are negative values present - jesli mamy orblem z asymetria danych to za pomoca logarytmu mozemy przejsc na liniowe i jest duzo lepsza interpretacja np przechodzimy od razu na interpetacje procentowa. lepsze bo wartosci nominalne sa czesto przeklamane. np w przypadku zarobkow w zal odinflacji etc wartosc nominalna znaczyc moze co innego, a procentowe wyrazenie zawsze odzwierciedla
(chipotle['item_price [$]'] - chipotle['item_price [$]'].min() + 1).transform(np.log)

### One-hot encoding (dummy variables) - zamienianie wartosci tekstowej na osobne kolumny . tworzymy sobie dodatkowe kolumny i dlaczego nie mamy kolumny dla ROme? bo jesli istambul i madryt jest 0 to wiadomo ze musi to byc Roma. wiec nie potrzebujemy miec az tylu wymiarow

| User   | City     | → | User | Istanbul | Madrid |
| ------ | ----     | --- | --- | --- | --- |
| 1      | Roma     | → | 1 | 0 | 0 |
| 2      | Madrid   | → | 2 | 0 | 1 |
| 1      | Madrid   | → | 1 | 0 | 1 |
| 3      | Istanbul | → | 3 | 1 | 0 |
| 2      | Istanbul | → | 2 | 1 | 0 |
| 1      | Istanbul | → | 1 | 1 | 0 |
| 1      | Roma     | → | 1 | 0 | 0 |



### GroupBy & apply

<img src="https://drive.google.com/uc?export=view&id=17TC0l9R_fE2-d932J6YILBCUu8pj_ox7" alt="Data processing: groupby" title="Data processing: groupby" align="left" width="600px" hspace="20px" vspace="20px"/>

In [ ]:
chipotle.item_name.unique()

In [ ]:
chipotle.head(10) # za duzo wiec wezmy pierwsze 10 obserwacji

In [ ]:
chipotle_10=chipotle.head(10)

In [ ]:
pd.get_dummies(chipotle_10['item_name']).astype(int) #(zeby zamiast false/true dawalo 0/1)

powyzej nie widac zeby ktorakolwiek kolumna miala same zera -> opcja dropfirst jest i wtedy mozemy wyrzucic kolumne ktorakolwiek, bo domyslnie bedziemy wiedzieli ze tam gdzie sa same 0 to ta kolumna ktorej nie ma.

In [ ]:
pd.get_dummies(chipotle_10['item_name'], drop_first=True).astype(int)

In [ ]:
chipotle.groupby('item_name').sum()[['quantity', 'item_price [$]']] # grupujemy i sumujemy sobie wszstkie wystapienia tych itemnamow i zliczamy quantity ile jest

In [ ]:
chipotle.groupby('item_name').sum()[['quantity', 'item_price [$]']] #dajemy wiecej niz jedna kolumne wiec dajemy zbior zmiennych, wiec gdybysmy zrobili tylko 1 nawias kwadratowy to by bylo nie jako zbior danych tylko pojedyncze wartosci. jesli mamy dwie kolumny to musi byc jako zbior danych. grupowanie juz bylo na zajeciach z Pandasa

### Split text

Useful when you have multiple traits in one column.

In [ ]:
chipotle['item_name'] # tu widzimy ze tekst podzielony spacjami

In [ ]:
chipotle['item_name'].str.split(' ') #wiec pierwsza rzecz ktora robimy to jak w kodzie nizej -> str ze tekst a potem split czyli podziel i na podst czego - w cudzyslow wrzucamy spacje

In [ ]:
chipotle['item_name'].str.split(' ').map(lambda x: x[0]) # gdy np chcemy z tekstu wydobyc tylko pierwszy człon - uzywamy funkcji indeksowania, wydobacz pierwszy element

In [ ]:
chipotle[chipotle['item_name'].str.startswith('Chicken')]['item_name'] #funkcja startswith od tego zeby zacelo nam od jkaiegos slowa

In [ ]:
chipotle[chipotle['item_name'].str.startswith('Chicken')]['item_name'].value_counts() #bardziej wyrafinowane podjecie - wez tylko te obiekty gdzie zaczyna sie od chicken i jeszcze pokaz ile kazdego jest

### Scaling - skalowanie wartosci - czyli wracamy do tematu wartosci ciaglych

In most cases, the numerical characteristics of a dataset are not specific and vary from one another. In fact, it doesn't make sense to expect the age and income columns to have the same range. But from a machine learning perspective, the same scope helps improve the model.

Scaling solves this problem. Continuous functions become range identical after the scaling process. This process is not mandatory for many algorithms, but still worthwhile. However, distance-based algorithms such as k-NN or k-Means must have scaled continuous functions as model inputs.
Basically, there are two common ways to scale: normalization and standardization.






In [ ]:
chipotle['item_price [$]'].describe() #w niektorych algorytmach jest tak wymagane ze trezba wartosci wystandaryzowac zeby byly z jakiegos zakresu wartosci, albo zeby sie zachowywaly w jakis sposob, bo tylko wtedy algorytm bedzie dobrze dzialaal

In [ ]:
# normalization - od kazdej wartosci odejmujemy wartosc minimalna a nast dzielimy przez roznice pom wartoscia max a minimalna. wzor na slacku na normalizacje
#doprowadzamy do sytuacji gdzie wszystkie wartosci sa z zakresu 0-1 czyli operujemy na % w sumie. to tez sposob na pozbycie sie wartosci odstajacych, ale lepiej najpierw sie tych odstajacych ozbyc bo inaczej itak bedzie nam zaburzac.

item_price_normalized = (chipotle['item_price [$]'] - chipotle['item_price [$]'].min()) / \
(chipotle['item_price [$]'].max() - chipotle['item_price [$]'].min())

In [ ]:
#kolejny sposob sklaowania wartosci to standaryzacja - wzor na standaryzacje jest na slacku
#x czyli wartosc, odejmujemy od tego srednia wartosc i dzielimy przez odchylenie standardowe

In [ ]:
item_price_normalized.describe()

In [ ]:
# standardization

item_price_standardized = (chipotle['item_price [$]'] - chipotle['item_price [$]'].mean()) / \
chipotle['item_price [$]'].std()

In [ ]:
item_price_standardized.describe() #tu mamy wartosci sprowadozne do rozkladu normalnego i mamy srednia bardzo blisko zeru. jest trohce asymetria lewostornna bo wartosc minimalna bezwgl. jest wieksze niz 1,22 max. mamy wystandaryzowany rozklad, zbierzny z rozkładem normalnym dzieki standaryzacji
#standaryzjaca wymagana czesto w wielu algorytmach bo radza sobie lepiej z wartosciami wystand. bo działamy na wartosciach z podobnych rozkladow. czyli rozne wartosci z roznych zmiennych ale z podobnych rozkladów

### Data extraction - wydobywanie dat, ekstrakcja dat. dosc wazne, czasem upierdliwe, trzeba pokombinowac, bo bardzo czesto mamy np dane rozlozone w czasie np dane giełdowe

In [ ]:
from datetime import date

In [ ]:
data = pd.DataFrame({
    'date': ['01-01-2017', '04-12-2008', '23-06-1988', '25-08-1999', '20-02-1993']
})

data

In [ ]:
data['date'] = pd.to_datetime(data.date, format="%d-%m-%Y") #zamienia nam konkretna kolumne ze zbioru danych date i zamienia nam te kolumne na typ Data. dodatkowo wczytujemy sobie format czyli mowimy ze pierwsze to dzien-%m miesiac-%Y rok z duzej litery  a małe y to format 24
#Y - format 2024
#y - format 24
# gdyby byl czas to najczesniej %H (czyli hours itd):%M:%S dwukropek zazwyczaj ale zwrocic uwage jak jest rozdzielony

In [17]:
data.dtypes

NameError: name 'data' is not defined

In [ ]:
data['date'].dt.year # tu mozemy z daty wydobyc tylko rok

In [ ]:
data['date'].dt.month #albo wydobyc sobie tylko. dzien

In [ ]:
data['date'].dt.day

In [ ]:
data['date'].dt.day_of_week #wydobycie info jaki to dzien tygodnia)

In [ ]:
data['passed_years'] = date.today().year - data['date'].dt.year
data['passed_years']

In [ ]:
data['passed_months'] = 12 * data['passed_years'] + date.today().month - data['date'].dt.month #ile minelo miesiecy
data['passed_months']

In [ ]:
data['day_name'] = data['date'].dt.day_name() # no i wydobycie nazwy dnia jaki byl wtedy
data['day_name']

### Feature engineering example - czyli wydobywanie, tworzenie nowych zmiennych z danych surowych

How to assess whether a given profile on Facebook is not a bot? - np chcemy sprawdzic czy dany profil jest botem czy nie, bo wtedy fb nic na tym nie zarabia bo generuje ruch ale wiadomo ze niczego nie kupi

* number of friends
* number of posts published
* number of photos
* number of likes
* likes under posts
* the length of the posts
* the origin of the phone number - bo sa kraje gdzie tych botow jest wiecej
* e-mail address domainbot nie dzialal po 5-6 iteracjach
* jak rusza myszka po ekranie czyli np czy uzytkownik skrolujac info rusza kursorem
*bot chodzacy postronie i zbierajacy dane o nich-> ustawic sie na stronie i przejechac do konca i wtedy bot byl w stanie zauwazyc wsystkie ogloszenia, ale po 2 latach juz

And the rating of the attractiveness of a movie on YouTube?



The car rental company ARRA wants to evaluate the reliability and maintenance costs of the different car models available in the fleet. It has the following data:
* a table with individual activities of all cars in the fleet: date, place of rental, status (driving, breakdown), renting data (age, sex, nationality), number of kilometers traveled, amount of fuel consumed, car identification number
* a table for each car: year of production, engine, equipment, brand, model, etc.


What features can we find here?






<img src="https://drive.google.com/uc?export=view&id=1UO2urRciECzoKE_vHy4RMGfFbkOWOGlW" alt="SDA logo" align="left" width="100px" hspace="10px" vspace="10px"/>
<br>

# TASKS

## **<font color='#306998'>Information for TASKs </font><font color='#ffd33b'>1-2</font>**

The following data was was made available on https://www.kaggle.com/datasets/davidbnn92/weather-data-for-covid19-data-analysis/.

Try downloading it from there using the `Download` button or with the API command `!kaggle datasets download -d davidbnn92/weather-data-for-covid19-data-analysis` (see https://www.kaggle.com/discussions/general/74235 for more details).

If the data is no longer available you can always download it from our Google Drive https://drive.google.com/drive/folders/1KXr6yUW7rE0LzUzuuMehEzam8D9zxGpv?usp=sharing.

### About Dataset

The dataset contains selected metereological features, such as temperature or wind speed, and was imported from the `NOAA GSOD dataset`, continuously updated to include recent measurments.

> Among others, you can find the following columns here:
* `Id`
* `Country/Region`
* `Date`
* `temp`: Mean temperature for the day in degrees Fahrenheit to tenths.
* `max`: Maximum temperature reported during the day.
* `min`: Minimum temperature reported during the day.
* `stp`: Mean station pressure for the day in millibars to tenths.
* `slp`: Mean sea level pressure for the day.
* `dewp`: Mean dew point for the day in [Fahrenheit to tenths].
* `wdsp`: Mean wind speed for the day in [knots to tenths].
* `prcp`: Total precipitation (rain and/or melted snow) reported during the day in [inches and hundredths]; `.00` indicates no measurable precipitation (includes a trace).
* `fog`: Indicators (1 = yes, 0 = no/not reported) for the occurrence during the day.</br></br>
Note that time of max/min temperatures varies by country and region, so this will sometimes not be the max for the calendar day.

## **<font color='#306998'>TASK </font><font color='#ffd33b'>1</font>**

Check how many rows contain missing data in `slp` column. Delete those rows and reset the rows indexes (see the `.reset_index()` method).


In [ ]:
import pandas as pd
df_weather=pd.read_csv('/content/training_data_with_weather_info_week_4 (1).csv')
df_weather=df_weather[['Id', 'Country_Region', 'Date',
       'temp', 'min', 'max', 'stp', 'slp', 'dewp', 'wdsp', 'prcp',
       'fog']] #chce miec tylko te kolumny id itd ktore mnie interesuja to przekopiowane z wczesiejszego zadnaia i tu  wzielam z czatu

In [ ]:
#1. sprawdz ile wierszy ma braki danych w kolumnie spl a najpierw wgl mozesz sprawdzic ile jest w calym zbiorze dla kazdej kolumeny
#2. usun wiersze gdzie slp ma brak danych
#3. zresetuj indeksy czyli zeby jak usuniemy wiersze to beda wyrywki wiec ma ladnie wygladac czyli od 0 do wartosci max bez zadnych dziur - to bylo na pandasie o resetowaniu indexow metoda reset_index()method

In [ ]:
#sprawdzam ile wgl mam danych
len(df_weather)

In [ ]:
#sprawdzam ile mam braków w wierszach w kolumnie spl
import numpy as np
df_weather.isna().sum()

In [ ]:
df_weather.isna().sum()/len(df_weather) #przechodzimy na procenty

In [ ]:
#usun wiersze gdzie slp ma brak danych
df_weather=df_weather[~df_weather['slp'].isnull()].reset_index()

In [ ]:
#usuwam kolumne z indexem

## **<font color='#306998'>TASK </font><font color='#ffd33b'>2</font>**

For the data frame from **Task 1**:
* generate missing data by removing the randomly selected 10% of values and replacing them with `-99` (e.g. use the following command `df_weather = df_weather.mask(np.random.random(df.shape) < .1)`)
* fill in the missing data with the
    * mean value
    * median value
    * mean value wrt. to Country (groupby first to get mean values)
* remove outlier observations
* enter a new column with discrete values for temperature (you can use the `cut()` or `qcut()` methods from the `pandas` library)
* enter the so-called *dummy variables* for those discrete temperature values
* use normalization or standardization to scale the temperature data
* create an additional `date` column containing the date in `pd.DateTime()` format

Tips:
* the missing data in the form `-99` can be replaced with something standard, e.g. `df_weather = df_weather.replace(-99, np.NaN)` then all standard methods such as: `.isna(), .isnull(), .fillna(), .dropna()` can be used
* to generate dummy variables, the so-called one-hot-encoding can be used:
    * `pd.get_dummies()` from pandas
    * `sklearn.preprocessing.OneHotEncoder` from scikit-learn
    * `tf.keras.layers.CategoryEncoding` from tensorflow
* data scaling can be done as in the slides or with the scikit-learn library, such as `sklearn.preprocessing.StandardScaler` or `sklearn.preprocessing.MinMaxScaler`


In [15]:
df_weather['temp'].fillna(df_weather['Country_Region'].map(df_weather.groupby('Country_Region')['temp'].mean()))

NameError: name 'df_weather' is not defined

In [16]:
#utworzenie wykresu

import seaborn as sns
ax=sns.histplot(data=df_weather,x='temp')
ax.axvline(df_weather['temp'].mean(),color='k',lw=2)
ax.axvline(df_weather['temp'].mean()-df_weather['temp'].std()*3,color='r',lw=1,ls='--')
ax.axvline(df_weather['temp'].mean()+df_weather['temp'].std()*3,color='r',lw=1,ls='--')
factor=3
temp_mean=df_weather['temp'].mean()
temp_std=df_weather['temp'].std()

upper_lim=temp_mean+temp_std*factor
lower_lim=temp_mean-temp_std*factor

mask=((df_weather['temp']<upper_lim) & (df_weather['temp']>lower_lim)) | (df_weather['temp'].isnull())

NameError: name 'df_weather' is not defined

In [24]:
#zdefiniuj ile ma byc binsow
pd.cut(df_weather['temp'],bins=10).value_counts()


NameError: name 'df_weather' is not defined

In [ ]:
pd.qcut(df_weather['temp'],bins=10).value_counts() #qcut dzieli nam na decyle jesli damy 10

In [23]:
import numpy as numpy